合并docx文档

In [2]:
import os
import re

import win32com.client as wc
import docx
from docxcompose.composer import Composer
import pandas as pd
import tqdm

In [3]:
# 收集所有路径
def get_path(root):
    res = list()
    info = os.walk(root)
    for tu in info:
        direc = tu[0]
        for file in tu[2]:
            path = os.path.join(direc, file)
            res.append(path)
    return res


# Convert doc to docx
def doc2docx(doc_path, docx_path):
    word = wc.Dispatch("Word.Application")
    doc = word.Documents.Open(doc_path)
    doc.SaveAs(docx_path, 12)
    doc.Close()
    word.Quit()
    

# 合并docx，返回Document实例
# 不能够保持格式
def merge_obj(paths):
    doc = docx.Document()
    for i, p in enumerate(paths):
        subdoc = docx.Document(p)
        
        # 最后一个文档不加page reak
        if i < len(paths) - 1:
            subdoc.add_page_break()
        
        for elem in subdoc.element.body:
            doc.element.body.append(elem)
    return doc


# 合并docx，保存至文件
def merge_docx(outpath, paths):
    doc = merge_obj(paths)
    doc.save(outpath)

    
# 合并docx，保存至文件。版本2
def merge_docx_v2(outpath, paths):
    print('要合并的文档数量：', len(paths))
    master = docx.Document(paths[0])
    composer = Composer(master)
    for p in tqdm.tqdm(paths[1:]):
        try:
            doc = docx.Document(p)
            # doc.add_page_break()
            composer.append(doc)
        except:
            print(p)
    composer.save(outpath)


# 合并docx，不断保存。版本3
def merge_docx_v3(outpath, paths):
    print('要合并的文档数量：', len(paths))
    
    # 先拷贝一份第一个文档
    obj = docx.Document(paths[0])
    obj.save(outpath)

    for p in tqdm.tqdm(paths[1:]):
        master = docx.Document(outpath)
        composer = Composer(master)
        doc = docx.Document(p)
        composer.append(doc)
        composer.save(outpath)
    

# 合并docx，输出信息
class Merger:
    def __init__(self, datadir, merged_path, info_path):
        self.datadir = datadir
        self.merged_path = merged_path
        self.info_path = info_path
        
        self.data_paths = get_path(self.datadir)
        self.metas = list()
        self.docxpaths = list()
    
    @staticmethod
    def get_path_suffix(p):
        _, suffix = os.path.splitext(p)
        return suffix
    
    @staticmethod
    def must_docx(p):
        suffix = Merger.get_path_suffix(p)
        if suffix == '.docx':
            return p
        elif suffix == '.doc':
            docxp = p + 'x'
            if os.path.exists(docxp):
                doc2docx(p, docxp)
            return docxp
        else:
            return None
                    
    def merge(self):
        docxpaths = list()
        # 收集docx路径和文件信息
        for p in tqdm.tqdm(self.data_paths):
            meta = dict()
            meta['file_path'] = p
            suffix = Merger.get_path_suffix(p)
            meta['suffix'] = suffix
            p = Merger.must_docx(p)
            if p and p not in docxpaths:
                docxpaths.append(p)
                meta['remark'] = '合并'
            else:
                meta['remark'] = ''
            self.metas.append(meta)
        # 合并
        self.docxpaths = docxpaths
        merge_docx_v2(self.merged_path, docxpaths)
        # 保存文件信息
        self.write_excel()
        
    def write_excel(self):
        df = pd.DataFrame(self.metas)
        df.to_excel(self.info_path, index=False, encoding='utf-8-sig')

In [4]:
# main
workdir = r'G:\ECPH_LY\Data\协助同事\刘艳'
# filedir = os.path.join(workdir, 'test')
# filedir = os.path.join(workdir, '文档')
# filedir = os.path.join(workdir, '202107-202109传统竹家具词条撰写')
# filedir = os.path.join(workdir, '家具场景（88条）')
filedir = os.path.join(workdir, '儿科护理（一审后清稿）')
outpath = os.path.join(workdir, '儿科护理审稿后.docx')
info_path = os.path.join(workdir, 'info.xlsx')

# 获取所有路径
paths = get_path(filedir)
print('File numbers: ', len(paths))

# 获取所有文件类型
suffix = [os.path.splitext(p)[1] for p in paths]
print(set(suffix))

File numbers:  20
{'.xlsx', '.docx'}


In [5]:
# 执行合并
merger = Merger(filedir, outpath, info_path)
merger.merge()

100%|███████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 20010.99it/s]


要合并的文档数量： 19


 50%|█████████████████████████████████████████▌                                         | 9/18 [00:00<00:00, 11.88it/s]

G:\ECPH_LY\Data\协助同事\刘艳\儿科护理（一审后清稿）\8.16 儿童和青少年常见心理及行为障碍患儿护理.docx


 83%|████████████████████████████████████████████████████████████████████▎             | 15/18 [00:01<00:00,  7.08it/s]

G:\ECPH_LY\Data\协助同事\刘艳\儿科护理（一审后清稿）\8.5 营养性疾病患儿护理.docx


 94%|█████████████████████████████████████████████████████████████████████████████▍    | 17/18 [00:02<00:00,  8.12it/s]

G:\ECPH_LY\Data\协助同事\刘艳\儿科护理（一审后清稿）\8.7 呼吸系统疾病患儿护理.docx


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:02<00:00,  8.34it/s]
C:\Users\dbk\anaconda3\Lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


新的任务：按顺序合并

In [10]:
# 利用继承
class SortMerger(Merger):
    def __init__(self, sort_func, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.sort_func = sort_func
    
    def merge(self):
        docxpaths = list()
        # 收集docx路径和文件信息
        for p in tqdm.tqdm(self.data_paths):
            meta = dict()
            meta['file_path'] = p
            suffix = Merger.get_path_suffix(p)
            meta['suffix'] = suffix
            p = Merger.must_docx(p)
            if p and p not in docxpaths:
                docxpaths.append(p)
                meta['remark'] = '合并'
            else:
                meta['remark'] = ''
            self.metas.append(meta)
        # 合并
        docxpaths = self.sort_func(docxpaths)  # 这里发生了变化
        self.docxpaths = docxpaths
        merge_docx_v2(self.merged_path, docxpaths)
        # 保存文件信息
        self.write_excel()

        
def get_head_digit(path):
    name = os.path.split(path)[1]
    pat = re.compile('^[\d]{1,5}')
    m = re.match(pat, name)
    if m:
        return int(m.group())
    return 1e10

def sort_func(lst):    
    r = sorted(lst, key=get_head_digit)
    return r

In [11]:
# main
workdir = r'G:\ECPH_LY\Data\协助同事\孙冬梅'
filedir = os.path.join(workdir, '第四批作家词条回稿文件')
outpath = os.path.join(workdir, '第四批作家词条回稿文件.docx')
info_path = os.path.join(workdir, 'info.xlsx')

# 获取所有路径
paths = get_path(filedir)
print('File numbers: ', len(paths))

# 获取所有文件类型
suffix = [os.path.splitext(p)[1] for p in paths]
print(set(suffix))

# 测试排序
# sorted_paths = sort_func(paths)
# for p in sorted_paths:
#     print(get_head_digit(p))
# print(sorted_paths)

File numbers:  201
{'.docx'}


In [12]:
# 执行合并
merger = SortMerger(sort_func, filedir, outpath, info_path)
merger.merge()

100%|█████████████████████████████████████████████████████████████████████████████| 201/201 [00:00<00:00, 67042.16it/s]


要合并的文档数量： 201


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:09<00:00, 22.10it/s]


新的任务：执行多个合并

In [9]:
# main
workdir = r'D:\workdir\一般性的文档处理\20220902-合并文档'
docx_root = os.path.join(workdir, '安然三审稿')
filedirs = [os.path.join(docx_root, d) for d in os.listdir(docx_root)]


for d in filedirs:
    outpath = os.path.join(docx_root, os.path.basename(d)+'.docx')
    info_path = os.path.join(docx_root, os.path.basename(d)+'_info.xlsx')

    # 获取所有路径
    paths = get_path(d)
    print('File numbers: ', len(paths))

    # 获取所有文件类型
    suffix = [os.path.splitext(p)[1] for p in paths]
    print(set(suffix))
    
    # 执行合并
    merger = Merger(d, outpath, info_path)
    merger.merge()

 21%|█████████████████▎                                                               | 18/84 [00:00<00:00, 175.18it/s]

File numbers:  85
{'.docx'}
要合并的文档数量： 85


 17%|█████████████▋                                                                   | 12/71 [00:00<00:00, 113.51it/s]

File numbers:  72
{'.docx'}
要合并的文档数量： 72


 13%|██████████▉                                                                       | 10/75 [00:00<00:00, 86.44it/s]

...

 10%|████████▏                                                                       | 13/127 [00:00<00:00, 116.71it/s]

File numbers:  128
{'.docx'}
要合并的文档数量： 128


 28%|██████████████████████▉                                                          | 15/53 [00:00<00:00, 137.98it/s]

File numbers:  54
{'.docx'}
要合并的文档数量： 54


100%|█████████████████████████████████████████████████████████████████████████████████| 53/53 [00:00<00:00, 110.25it/s]


以下为测试：

In [15]:
for i in paths:
    _, fname = os.path.split(i)
    print(fname)

万云骏.docx
何宋苏.docx
何静源.docx
俞樾.docx
俞程競英.docx
傅润森.docx
刘楚青.docx
包丹庭.docx
史溥泉.docx
叶仰曦.docx
叶堂.docx
叶小纨.docx
叶惠农.docx
吴江沈氏家族.docx
吴粹伦.doc
吴粹伦.docx
吴鸿迈.docx
周妙中.docx
周铨庵.docx
唐圭璋.docx
唐文治.docx
夏煥新.docx
孙天申.docx
庄一拂.docx
张允和.docx
张元和.docx
张充和.docx
张厚衡(4)(1).docx
张厚衡.docx
张善芗.docx
张宗和.docx
张琦翔.docx
张荫朗 刘珏.docx
張麗真.docx
徐大椿.docx
徐炎之.docx
徐燨.docx
徐爔.docx
徐致靖.docx
戴俊.docx
戴夏.docx
朱再舫.docx
朱家溍.docx
朱尧亭.docx
朱尧文.docx
朱復.docx
朱经畬.docx
朱经畲.docx
杨忞.docx
林焘.docx
柳萱图.docx
楼宇烈.docx
樊书培.docx
樊伯炎.docx
樊诵芬.docx
樊颖初.docx
殷菊侬.docx
殷震贤.docx
汪健君.docx
汪小丹.docx
汪鼎丞.docx
沈化中.docx
沈宠绥.docx
沈宪.docx
沈永乔.docx
沈永令.docx
沈珂.docx
沈璟.docx
沈瓒.docx
沈自南.docx
沈自友.docx
沈自徵.docx
沈自昌.docx
沈自晋.docx
沈自普.docx
沈自炳.docx
沈自继.docx
焦承允.docx
爱新觉罗毓婍.docx
王西徵.docx
王颂椒.docx
甘南轩.docx
甘律之.docx
甘纹轩.docx
甘贡三.docx
甘长华.docx
瞿松涛.docx
祝宽.docx
穆藕初.docx
章元善.docx
童斐.docx
童曼秋.docx
管际安.docx
肖漪.docx
范崇实.docx
蒋复璁.docx
蔡安安.docx
袁敏宣.docx
许宝騋.docx
许淑春.docx
许潜庵.doc
许潜庵.docx
许雨香.docx
许鸿宾.docx
谢锡恩.docx
贝祖武.docx
赵子敬.docx
赵景深.docx
邵怀民.docx
钱一羽.docx
陆剑霞.docx
陆坤.